### Dependencies 

In [16]:
import pandas as pd
from rouge_score import rouge_scorer

In [19]:
df_without_domain = pd.read_csv("eval_df_preds_without_domain.csv")
df_with_domain = pd.read_csv("eval_df_preds_with_domain.csv")

In [20]:
df_without_domain.head()

,Unnamed: 0,target_text,input_text,prefix,predicted_titles
0,0,User Perceptions of Smart Home IoT Privacy,Smart home Internet of Things (IoT) devices ...,summarize,Semi-structured interviews with smart home owners
1,1,Weak Cosmic Censorship: As Strong as Ever,Spacetime singularities that arise in gravit...,summarize,The cosmic censorship conjecture
2,2,Search for first-generation scalar leptoquarks...,We report on a search for pair production of...,summarize,Search for pair production of first-generation...
3,3,New scalar field models and their defect solut...,In this work we introduce new scalar field m...,summarize,Deformation procedure and new scalar field models
4,4,Curvature quintessence matched with observatio...,Quintessence issues can be achieved by takin...,summarize,Cosmological solutions for higher order curvat...


In [21]:
df_with_domain.head()

,target_text,input_text,prefix,predicted_titles
0,User Perceptions of Smart Home IoT Privacy,Smart home Internet of Things (IoT) devices ...,summarize,Semi-structured interviews with smart home owners
1,Weak Cosmic Censorship: As Strong as Ever,Spacetime singularities that arise in gravit...,summarize,The weak cosmic censorship conjecture
2,Search for first-generation scalar leptoquarks...,We report on a search for pair production of...,summarize,Search for pair production of first-generation...
3,New scalar field models and their defect solut...,In this work we introduce new scalar field m...,summarize,Deformation procedure and new scalar field models
4,Curvature quintessence matched with observatio...,Quintessence issues can be achieved by takin...,summarize,Cosmological solutions for higher order curvat...


In [25]:
references = list(df_without_domain.target_text)
candidates_with_domain = list(df_with_domain.predicted_titles)
candidates_without_domain = list(df_without_domain.predicted_titles)

In [27]:
for x, y, z in zip(references, candidates_with_domain, candidates_without_domain):
    print(x, y, z)
    break

User Perceptions of Smart Home IoT Privacy Semi-structured interviews with smart home owners Semi-structured interviews with smart home owners


In [24]:
references

['This is a test.', 'This is a test.', 'This is a test.']

In [29]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rouge3', 'rougeL'], use_stemmer=True)

In [34]:
for x, y, z in zip(references, candidates_with_domain, candidates_without_domain):
    print(scorer.score(x, y)['rouge1'].preci)
    print(scorer.score(x, z))
    break

<class 'rouge_score.scoring.Score'>
{'rouge1': Score(precision=0.2857142857142857, recall=0.2857142857142857, fmeasure=0.2857142857142857), 'rouge2': Score(precision=0.16666666666666666, recall=0.16666666666666666, fmeasure=0.16666666666666666), 'rouge3': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.2857142857142857, recall=0.2857142857142857, fmeasure=0.2857142857142857)}


In [37]:
from PyRouge.pyrouge import Rouge

In [38]:
r = Rouge()

In [39]:
[precision, recall, f_score] = r.rouge_l([references], [candidates_with_domain])

In [40]:
precision

0.001806083650190114

In [41]:
recall

0.001806083650190114

In [42]:
f_score

0.001807033651574286